
FOR THE JSON  DATA: 20230916_01-16-54-377_1018_userinfo.json. 

{"CGUID":"1018","Date":"20230916_01:16:54:377",

"ShotResult":[

    7.6970219612121582, 25.65673828125, -----> Head speed (m/s)  is sum of first and second column = 7.697021961212158 + 25.65673828125

    -0.608303964138031, ------> Face angle (degree)

    8.4586247339757392E-07, -----> Attack angle (degree)

    -2.7872023582458496, -----> Swing path (degree)

    2.0588235855102539, ------> Swing tempo (ratio)

    97.855049133300781,  -----> Swing backtop (degree) 

    0.34999999403953552, -----> Back swing time (second) 

    0.17000000178813934, -----> Down swing time (second)

    1.0 -----> Hand (1 = RIGHT, 2 = LEFT) 

In [25]:
import os
import pandas as pd
import json
import numpy as np

# File directories 
input_directory = r'C:\Users\james\Desktop\2023-2\Raw\logRaws'
output_directory = r'C:\Users\James\Desktop\2023-2\Raw\userInfo'

# Preprocess data for a single file
def preprocess_single_file(input_file_path, output_file_path):

    # Load CSV data -> filter rows where Swing Stage is not 0 -> extract swing_stage + 3 accel + 3 gyro values
    input_data = pd.read_csv(input_file_path)
    input_data = input_data[input_data.iloc[:, 3] != 0]
    input_data = input_data.iloc[:, [3, 19, 20, 21, 25, 26, 27]]
    # print(input_data.shape)
    # input_data.columns = ['swing_stage', 'accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']


    # Pad remaining values with zeros -> return np.array 
    target_shape = (999,7)
    input_values = input_data.values
    pad_rows = max(0, target_shape[0] - input_values.shape[0])
    pad_cols = max(0, target_shape[1] - input_values.shape[1])
    X = np.pad(input_values, ((0, pad_rows), (0, pad_cols)), mode='constant')
    # print(X)

    # Load JSON data -> If output is not available, skip
    with open(output_file_path, 'r') as json_file:
        try:
            output_data = json.load(json_file)
            output_data_df = pd.DataFrame(output_data['ShotResult'])
            output_data_df = pd.DataFrame.transpose(output_data_df)

            columns_to_remove = [5,7,8,9]
            output_data_df = output_data_df.drop(columns=columns_to_remove)
            # output_data_df.columns = ['swingspeed1', 'swingspeed2', 'faceangle', 'attackangle', 'swingpath', 'swingbacktop']
            # print(output_data_df.shape)
            Y = np.array(output_data_df)
            # print(Y)
        except KeyError as e:
            return None, None

    return X, Y


def preprocess_all_files():

    X_all_data = []
    Y_all_data = []

    # Iterate through all files
    for input_file in os.listdir(input_directory):

        # Construct the output file path based on the common prefix
        common_prefix = '_'.join(input_file.split('_')[:-1]) + '_'
        output_file = common_prefix + 'userinfo.json'
        output_file_path = os.path.join(output_directory, output_file)

        # Construct the input file path
        input_file_path = os.path.join(input_directory, input_file)

        # Preprocess the data for the current file
        X, Y = preprocess_single_file(input_file_path, output_file_path)
    
        # Append the preprocessed data to the list if both values exist
        if X is not None and Y is not None:
            X_all_data.append(X)
            Y_all_data.append(Y)


    return np.array(X_all_data), np.array(Y_all_data)

In [34]:
import tensorflow as tf

def percentage_difference_loss(y_true, y_pred):
    # Calculate percentage difference
    percentage_diff = tf.abs((y_true - y_pred) / tf.maximum(tf.abs(y_true), tf.keras.backend.epsilon()))
    
    # Take the mean across all elements in the batch
    mean_percentage_diff = tf.reduce_mean(percentage_diff)
    
    return mean_percentage_diff

In [35]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
import pandas as pd

X_data, Y_data  = preprocess_all_files()

# Check input and output shape
print(type(X_data))
print(X_data.shape)
print(type(Y_data))
print(Y_data.shape)

# Split data
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=0.2, random_state=42)

# Define model
model = Sequential()
model.add(LSTM(64, input_shape=(999, 7), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(6, activation='linear'))

# Compile modle
model.compile(optimizer='adam', loss=percentage_difference_loss)
model.summary()

# Train model
model.fit(X_train, Y_train, epochs=50, batch_size=32) 

# Evaluate model
score = model.evaluate(X_test, Y_test)
print(score)

tf.saved_model.save(model, r'C:\Users\james\Desktop\2023-2\RNN')

<class 'numpy.ndarray'>
(3823, 999, 7)
<class 'numpy.ndarray'>
(3823, 1, 6)
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 999, 64)           18432     
                                                                 
 dropout_12 (Dropout)        (None, 999, 64)           0         
                                                                 
 lstm_13 (LSTM)              (None, 64)                33024     
                                                                 
 dropout_13 (Dropout)        (None, 64)                0         
                                                                 
 dense_6 (Dense)             (None, 6)                 390       
                                                                 
Total params: 51846 (202.52 KB)
Trainable params: 51846 (202.52 KB)
Non-trainable params: 0 (0.00 Byte)
______

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
96/96 [==============================] - 74s 712ms/step - loss: 2.2003
Epoch 2/50
96/96 [==============================] - 76s 791ms/step - loss: 1.2566
Epoch 3/50
96/96 [==============================] - 47s 490ms/step - loss: 1.1939
Epoch 4/50
96/96 [==============================] - 35s 368ms/step - loss: 0.7984
Epoch 5/50
96/96 [==============================] - 34s 354ms/step - loss: 0.7565
Epoch 6/50
96/96 [==============================] - 34s 352ms/step - loss: 0.6179
Epoch 7/50
96/96 [==============================] - 34s 355ms/step - loss: 0.5230
Epoch 8/50
96/96 [==============================] - 34s 357ms/step - loss: 0.5018
Epoch 9/50
9

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
24/24 [==============================] - 3s 111ms/step - loss: 0.1950
0.19498607516288757
INFO:tensorflow:Assets written to: C:\Users\james\Desktop\2023-2\RNN\assets


INFO:tensorflow:Assets written to: C:\Users\james\Desktop\2023-2\RNN\assets


In [ ]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

def model_builder(hp):
    model = keras.Sequential()

    model.add(keras.layers.LSTM(units=hp.Int('units', min_value=32, max_value=512, step=32),
                                input_shape=(999, 7)))

    model.add(keras.layers.Dense(units=6, activation='linear'))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=percentage_difference_loss,
                  metrics=['accuracy'])

    return model